### Deep Learning
#### Multi-layer perceptron (MLP) for regression (from scratch)
A Multilayer Perceptron (MLP) for regression is a feedforward neural network with:
- Input Layer: Receives feature vectors.
- Hidden Layers: Use nonlinear activations (e.g., ReLU) to learn patterns.
- Output Layer: Single or multiple neurons with linear activations to predict continuous values.
- Loss Function: Here, the Mean Squared Error (MSE) to minimize prediction errors.

The pseudo-code of training a three-layer MLP (generally any MLP) for regression with **backpropagation** is specified as:     
<br>1. **Initialize** weights $W_1$,$b_1$,$W_2$,$b_2$.
<br>2. For each epoch:
 - **Shuffle** training data.
 - **Split** into mini-batches of size batch_size.
 - For each mini-batch:
   - **Forward pass:** Compute predictions $A_2$.
   - **Compute loss:** MSE using $Y−A_2$.
   - **Backward pass:** Compute gradients.
   - **Update weights:** With gradient descent.
3. **Evaluate** on validation set periodically.

Here, $A_2$ is the output of the output layer. $Y$ is the matrix of desired output such that each row $i$ is the desired vector $\boldsymbol{y}_i$ for the the input vector $\boldsymbol{x}_i$. We assume that we are given $n$ data pairs $(\boldsymbol{x}_i,\boldsymbol{y}_i)$ 

**Hint:** For the hidden layer, we use activation functions sush as: *Logistic*, *ReLU*, and *Tanh*.

<hr>
In the following, we implement a three-layer MLP for regression using a matrix form of the backpropagation with mini-batch. The whole formulae are given in the link below:
<hr>
https://github.com/ostad-ai/Machine-Learning
<br> Explanation: https://www.pinterest.com/HamedShahHosseini/Machine-Learning

In [1]:
# Import required modules
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# The class that implements the MLP with three layers for regression
class MLPRegressor:
    def __init__(self, input_size, hidden_size, output_size):
        # He initialization for ReLU
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
        self.b2 = np.zeros((1, output_size))

    # Activation function ReLU    
    def relu(self, x):
        return np.maximum(0, x)

    # Derivative of ReLU
    def relu_derivative(self, x):
        return (x > 0).astype(float)
    
    # Forward pass
    def forward(self, X):
        self.Z1 = X @ self.W1 + self.b1
        self.A1 = self.relu(self.Z1)
        self.Z2 = self.A1 @ self.W2 + self.b2
        self.A2 = self.Z2  # Linear activation
        return self.A2

    # Compute Mean Squared Error (MSE)
    def mse_loss(self, y_pred, y_true):
        return np.mean((y_true - y_pred) ** 2)  # y_true first!
    
    # Backward pass
    def backward(self, X, y, learning_rate):
        n = X.shape[0]

        # Output layer gradients (y_true - y_pred)
        dZ2 = -(y - self.A2)  
        dW2 = (1 / n) * (self.A1.T @ dZ2)
        db2 = (1 / n) * np.sum(dZ2, axis=0, keepdims=True)

        # Hidden layer gradients
        dA1 = dZ2 @ self.W2.T
        dZ1 = dA1 * self.relu_derivative(self.Z1)
        dW1 = (1 / n) * (X.T @ dZ1)
        db1 = (1 / n) * np.sum(dZ1, axis=0, keepdims=True)

        # Update parameters
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        
    # Train with with mini-batch and backpropagation
    def train(self, X, y, epochs, learning_rate, batch_size, X_val=None, y_val=None):
        n_samples = X.shape[0]
        for epoch in range(epochs):
            # Shuffle data
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            # Mini-batch training
            for i in range(0, n_samples, batch_size):
                X_batch = X_shuffled[i:i + batch_size]
                y_batch = y_shuffled[i:i + batch_size]
                self.forward(X_batch)
                self.backward(X_batch, y_batch, learning_rate)

            # Log progress
            if epoch % 10 == 0:
                train_pred = self.forward(X)
                train_loss = self.mse_loss(train_pred, y)
                if X_val is not None:
                    val_pred = self.forward(X_val)
                    val_loss = self.mse_loss(val_pred, y_val)
                    print(f"Epoch {epoch}, Train MSE: {train_loss:.4f}, Val MSE: {val_loss:.4f}")
                else:
                    print(f"Epoch {epoch}, Train MSE: {train_loss:.4f}")
                    
    # Compute output for the input matrix X 
    def predict(self, X):
        return self.forward(X)

In [3]:
# Example usage
if __name__ == "__main__":
    X, y = make_regression(n_samples=1000, n_features=10, n_targets=1, noise=0.1, random_state=42)
    y = y.reshape(-1, 1)  # Ensure y is 2D
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    mlp = MLPRegressor(input_size=10, hidden_size=64, output_size=1)
    mlp.train(X_train, y_train, epochs=100, learning_rate=0.01, batch_size=32, X_val=X_val, y_val=y_val)

    y_pred = mlp.predict(X_val)
    final_mse = mean_squared_error(y_val, y_pred)  # sklearn uses (y_true, y_pred)
    print(f"\nFinal Validation MSE: {final_mse:.4f}")

Epoch 0, Train MSE: 871.5028, Val MSE: 908.2278
Epoch 10, Train MSE: 7.1552, Val MSE: 9.6900
Epoch 20, Train MSE: 9.9743, Val MSE: 13.0336
Epoch 30, Train MSE: 1.1262, Val MSE: 2.1430
Epoch 40, Train MSE: 0.3651, Val MSE: 1.3042
Epoch 50, Train MSE: 0.3173, Val MSE: 1.1293
Epoch 60, Train MSE: 0.3466, Val MSE: 1.0649
Epoch 70, Train MSE: 0.6222, Val MSE: 1.3919
Epoch 80, Train MSE: 0.9323, Val MSE: 1.5319
Epoch 90, Train MSE: 0.2779, Val MSE: 1.0138

Final Validation MSE: 0.8744
